<a href="https://colab.research.google.com/github/tomohitom/Tennis_Game_Analysis/blob/master/Openpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/YutaroOgawa/pytorch_advanced.git"

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 529, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 529 (delta 23), reused 32 (delta 13), pack-reused 479
Receiving objects: 100% (529/529), 17.57 MiB | 29.94 MiB/s, done.
Resolving deltas: 100% (282/282), done.


In [ ]:
import os
import urllib.request
import zipfile
import tarfile

data_dir = "/content/pytorch_advanced/4_pose_estimation/data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

weights_dir = "/content/pytorch_advanced/4_pose_estimation/weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

url =  "http://images.cocodataset.org/zips/val2014.zip"
target_path = os.path.join(data_dir, "val2014.zip") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    zip = zipfile.ZipFile(target_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

COCO.jsonとmask.tar.gzを「data」ディレクトリにアップロード

In [ ]:
save_path = os.path.join("/content/pytorch_advanced/4_pose_estimation/data", "mask.tar.gz") 

with tarfile.open(save_path, 'r:*') as tar:
    tar.extractall("/content/pytorch_advanced/4_pose_estimation/data")

画像、マスク画像、アノテーションデータへのファイルパスリストを作成

In [ ]:
import json
import os
import numpy as np
import cv2
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

import torch.utils.data as data

def make_datapath_list(rootpath):
    json_path = os.path.join(rootpath, "COCO.json")
    with open(json_path) as data_file:
        data_this = json.load(data_file)
        data_json = data_this["root"]

    num_samples = len(data_json)
    train_indexes = []
    val_indexes = []
    for count in range(num_samples):
        if data_json[count]["isValidation"] != 0.:
            val_indexes.append(count)
        else:
            train_indexes.append(count)

    train_img_list = list()
    val_img_list = list()

    for idx in train_indexes:
        img_path = os.path.join(rootpath, data_json[idx]["img_paths"])
        train_img_list.append(img_path)

    for idx in val_indexes:
        img_path = os.path.join(rootpath, data_json[idx]["img_paths"])
        val_img_list.append(img_path)

    train_mask_list = []
    val_mask_list = []

    for idx in train_indexes:
        img_idx = data_json[idx]['img_paths'][-16:-4]
        anno_path = "./data/mask/train2014/mask_COCO_tarin2014_" + img_idx+'.jpg'
        train_mask_list.append(anno_path)

    for idx in val_indexes:
        img_idx = data_json[idx]['img_paths'][-16:-4]
        anno_path = "./data/mask/val2014/mask_COCO_val2014_" + img_idx+'.jpg'
        val_mask_list.append(anno_path)

    train_meta_list = list()
    val_meta_list = list()

    for idx in train_indexes:
        train_meta_list.append(data_json[idx])

    for idx in val_indexes:
        val_meta_list.append(data_json[idx])

    return train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list

In [ ]:
# 動作確認（実行には10秒ほど時間がかかります）
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(
    rootpath="./data/")

val_meta_list[24]

In [ ]:
index = 24

img = cv2.imread(val_img_list[index])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

mask_miss = cv2.imread(val_mask_list[index])
mask_miss = cv2.cvtColor(mask_miss, cv2.COLOR_BGR2RGB)
plt.imshow(mask_miss)
plt.show()

blend_img = cv2.addWeighted(img, 0.4, mask_miss, 0.6, 0)
plt.imshow(blend_img)
plt.show()

画像の前処理作成

In [ ]:
import cv2
import numpy as np
import random

import torch
from torchvision import transforms


class Compose(object):
    """引数transformに格納された変形を順番に実行するクラス
       対象画像、マスク画像、アノテーション画像を同時に変換させます。 
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, meta_data, img, mask_miss):
        for t in self.transforms:
            meta_data, img, mask_miss = t(meta_data, img, mask_miss)

        return meta_data, img, mask_miss


class get_anno(object):
    """JSON形式のアノテーションデータを辞書オブジェクトに格納"""

    def __call__(self, meta_data, img, mask_miss):
        anno = dict()
        anno['dataset'] = meta_data['dataset']
        anno['img_height'] = int(meta_data['img_height'])
        anno['img_width'] = int(meta_data['img_width'])

        anno['isValidation'] = meta_data['isValidation']
        anno['people_index'] = int(meta_data['people_index'])
        anno['annolist_index'] = int(meta_data['annolist_index'])

        # (b) objpos_x (float), objpos_y (float)
        anno['objpos'] = np.array(meta_data['objpos'])
        anno['scale_provided'] = meta_data['scale_provided']
        anno['joint_self'] = np.array(meta_data['joint_self'])

        anno['numOtherPeople'] = int(meta_data['numOtherPeople'])
        anno['num_keypoints_other'] = np.array(
            meta_data['num_keypoints_other'])
        anno['joint_others'] = np.array(meta_data['joint_others'])
        anno['objpos_other'] = np.array(meta_data['objpos_other'])
        anno['scale_provided_other'] = meta_data['scale_provided_other']
        anno['bbox_other'] = meta_data['bbox_other']
        anno['segment_area_other'] = meta_data['segment_area_other']

        if anno['numOtherPeople'] == 1:
            anno['joint_others'] = np.expand_dims(anno['joint_others'], 0)
            anno['objpos_other'] = np.expand_dims(anno['objpos_other'], 0)

        meta_data = anno

        return meta_data, img, mask_miss


class add_neck(object):
    '''
    アノテーションデータの順番を変更し、さらに首のアノテーションデータを追加します。
    首ポジションは両肩の位置から計算します。

    MS COCO annotation order:
    0: nose	   		1: l eye		2: r eye	3: l ear	4: r ear
    5: l shoulder	6: r shoulder	7: l elbow	8: r elbow
    9: l wrist		10: r wrist		11: l hip	12: r hip	13: l knee
    14: r knee		15: l ankle		16: r ankle
    The order in this work:
    (0-'nose'	1-'neck' 2-'right_shoulder' 3-'right_elbow' 4-'right_wrist'
    5-'left_shoulder' 6-'left_elbow'	    7-'left_wrist'  8-'right_hip'
    9-'right_knee'	 10-'right_ankle'	11-'left_hip'   12-'left_knee'
    13-'left_ankle'	 14-'right_eye'	    15-'left_eye'   16-'right_ear'
    17-'left_ear' )
    '''

    def __call__(self, meta_data, img, mask_miss):
        meta = meta_data
        our_order = [0, 17, 6, 8, 10, 5, 7, 9,
                     12, 14, 16, 11, 13, 15, 2, 1, 4, 3]
        # Index 6 is right shoulder and Index 5 is left shoulder
        right_shoulder = meta['joint_self'][6, :]
        left_shoulder = meta['joint_self'][5, :]
        neck = (right_shoulder + left_shoulder) / 2

        # right_shoulder[2]が値1のときはアノテーションがあり画像内に部位も見えている
        # 値0のときはアノテーションの座標情報はあるが、画像内に部位は映っていない
        # 値が2のときは画像内に写っておらず、アノテーション付けもない
        # ※注意　元のMSCOCOの定義と値の意味が変わっている
        # v=0: not labeled (in which case x=y=0), v=1: labeled but not visible, and v=2: labeled and visible.
        if right_shoulder[2] == 2 or left_shoulder[2] == 2:
            neck[2] = 2
        elif right_shoulder[2] == 1 or left_shoulder[2] == 1:
            neck[2] = 1
        else:
            neck[2] = right_shoulder[2] * left_shoulder[2]

        neck = neck.reshape(1, len(neck))
        neck = np.round(neck)
        meta['joint_self'] = np.vstack((meta['joint_self'], neck))
        meta['joint_self'] = meta['joint_self'][our_order, :]
        temp = []

        for i in range(meta['numOtherPeople']):
            right_shoulder = meta['joint_others'][i, 6, :]
            left_shoulder = meta['joint_others'][i, 5, :]
            neck = (right_shoulder + left_shoulder) / 2
            if (right_shoulder[2] == 2 or left_shoulder[2] == 2):
                neck[2] = 2
            elif (right_shoulder[2] == 1 or left_shoulder[2] == 1):
                neck[2] = 1
            else:
                neck[2] = right_shoulder[2] * left_shoulder[2]
            neck = neck.reshape(1, len(neck))
            neck = np.round(neck)
            single_p = np.vstack((meta['joint_others'][i], neck))
            single_p = single_p[our_order, :]
            temp.append(single_p)
        meta['joint_others'] = np.array(temp)

        meta_data = meta

        return meta_data, img, mask_miss


class aug_scale(object):
    def __init__(self):
        self.params_transform = dict()
        self.params_transform['scale_min'] = 0.5
        self.params_transform['scale_max'] = 1.1
        self.params_transform['target_dist'] = 0.6

    def __call__(self, meta_data, img, mask_miss):

        # ランダムに0.5倍～1.1倍する
        dice = random.random()  # (0,1)
        scale_multiplier = (
            self.params_transform['scale_max'] - self.params_transform['scale_min']) * dice + self.params_transform['scale_min']

        scale_abs = self.params_transform['target_dist'] / \
            meta_data['scale_provided']
        scale = scale_abs * scale_multiplier
        img = cv2.resize(img, (0, 0), fx=scale, fy=scale,
                         interpolation=cv2.INTER_CUBIC)

        mask_miss = cv2.resize(mask_miss, (0, 0), fx=scale,
                               fy=scale, interpolation=cv2.INTER_CUBIC)

        # modify meta data
        meta_data['objpos'] *= scale
        meta_data['joint_self'][:, :2] *= scale
        if (meta_data['numOtherPeople'] != 0):
            meta_data['objpos_other'] *= scale
            meta_data['joint_others'][:, :, :2] *= scale
        return meta_data, img, mask_miss


class aug_rotate(object):
    def __init__(self):
        self.params_transform = dict()
        self.params_transform['max_rotate_degree'] = 40

    def __call__(self, meta_data, img, mask_miss):

        # ランダムに-40～40度回転
        dice = random.random()  # (0,1)
        degree = (dice - 0.5) * 2 * \
            self.params_transform['max_rotate_degree']  # degree [-40,40]

        def rotate_bound(image, angle, bordervalue):
            # grab the dimensions of the image and then determine the
            # center
            (h, w) = image.shape[:2]
            (cX, cY) = (w // 2, h // 2)

            # grab the rotation matrix (applying the negative of the
            # angle to rotate clockwise), then grab the sine and cosine
            # (i.e., the rotation components of the matrix)
            M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
            cos = np.abs(M[0, 0])
            sin = np.abs(M[0, 1])

            # compute the new bounding dimensions of the image
            nW = int((h * sin) + (w * cos))
            nH = int((h * cos) + (w * sin))

            # adjust the rotation matrix to take into account translation
            M[0, 2] += (nW / 2) - cX
            M[1, 2] += (nH / 2) - cY

            # perform the actual rotation and return the image
            return cv2.warpAffine(image, M, (nW, nH), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT,
                                  borderValue=bordervalue), M

        def rotatepoint(p, R):
            point = np.zeros((3, 1))
            point[0] = p[0]
            point[1] = p[1]
            point[2] = 1

            new_point = R.dot(point)

            p[0] = new_point[0]

            p[1] = new_point[1]
            return p

        # 画像とマスク画像の回転
        img_rot, R = rotate_bound(img, np.copy(
            degree), (128, 128, 128))  # 回転でできた隙間は青色
        mask_miss_rot, _ = rotate_bound(
            mask_miss, np.copy(degree), (255, 255, 255))

        # アノテーションデータの回転
        meta_data['objpos'] = rotatepoint(meta_data['objpos'], R)

        for i in range(18):
            meta_data['joint_self'][i, :] = rotatepoint(
                meta_data['joint_self'][i, :], R)

        for j in range(meta_data['numOtherPeople']):

            meta_data['objpos_other'][j, :] = rotatepoint(
                meta_data['objpos_other'][j, :], R)

            for i in range(18):
                meta_data['joint_others'][j, i, :] = rotatepoint(
                    meta_data['joint_others'][j, i, :], R)

        return meta_data, img_rot, mask_miss_rot


class aug_croppad(object):
    def __init__(self):
        self.params_transform = dict()
        self.params_transform['center_perterb_max'] = 40
        self.params_transform['crop_size_x'] = 368
        self.params_transform['crop_size_y'] = 368

    def __call__(self, meta_data, img, mask_miss):

        # ランダムにオフセットを用意 -40から40
        dice_x = random.random()  # (0,1)
        dice_y = random.random()  # (0,1)
        crop_x = int(self.params_transform['crop_size_x'])
        crop_y = int(self.params_transform['crop_size_y'])
        x_offset = int((dice_x - 0.5) * 2 *
                       self.params_transform['center_perterb_max'])
        y_offset = int((dice_y - 0.5) * 2 *
                       self.params_transform['center_perterb_max'])

        center = meta_data['objpos'] + np.array([x_offset, y_offset])
        center = center.astype(int)

        # pad up and down
        # img.shape=（幅、高さ）
        pad_v = np.ones((crop_y, img.shape[1], 3), dtype=np.uint8) * 128
        pad_v_mask_miss = np.ones(
            (crop_y, mask_miss.shape[1], 3), dtype=np.uint8) * 255
        img = np.concatenate((pad_v, img, pad_v), axis=0)

        mask_miss = np.concatenate(
            (pad_v_mask_miss, mask_miss, pad_v_mask_miss), axis=0)

        # pad right and left
        # img.shape=（幅、高さ）
        pad_h = np.ones((img.shape[0], crop_x, 3), dtype=np.uint8) * 128
        pad_h_mask_miss = np.ones(
            (mask_miss.shape[0], crop_x, 3), dtype=np.uint8) * 255

        img = np.concatenate((pad_h, img, pad_h), axis=1)
        mask_miss = np.concatenate(
            (pad_h_mask_miss, mask_miss, pad_h_mask_miss), axis=1)

        # 切り出す
        img = img[int(center[1] + crop_y / 2):int(center[1] + crop_y / 2 + crop_y),
                  int(center[0] + crop_x / 2):int(center[0] + crop_x / 2 + crop_x), :]

        mask_miss = mask_miss[int(center[1] + crop_y / 2):int(center[1] + crop_y / 2 +
                                                              crop_y + 1*0), int(center[0] + crop_x / 2):int(center[0] + crop_x / 2 + crop_x + 1*0)]

        offset_left = crop_x / 2 - center[0]
        offset_up = crop_y / 2 - center[1]

        offset = np.array([offset_left, offset_up])
        meta_data['objpos'] += offset
        meta_data['joint_self'][:, :2] += offset

        # 画像からはみ出ていないかチェック
        # 条件式4つのORを計算する
        mask = np.logical_or.reduce((meta_data['joint_self'][:, 0] >= crop_x,
                                     meta_data['joint_self'][:, 0] < 0,
                                     meta_data['joint_self'][:, 1] >= crop_y,
                                     meta_data['joint_self'][:, 1] < 0))

        meta_data['joint_self'][mask == True, 2] = 2
        if (meta_data['numOtherPeople'] != 0):
            meta_data['objpos_other'] += offset
            meta_data['joint_others'][:, :, :2] += offset

            # 条件式4つのORを計算する
            mask = np.logical_or.reduce((meta_data['joint_others'][:, :, 0] >= crop_x,
                                         meta_data['joint_others'][:,
                                                                   :, 0] < 0,
                                         meta_data['joint_others'][:,
                                                                   :, 1] >= crop_y,
                                         meta_data['joint_others'][:, :, 1] < 0))

            meta_data['joint_others'][mask == True, 2] = 2

        return meta_data, img, mask_miss


class aug_flip(object):
    def __init__(self):
        self.params_transform = dict()
        self.params_transform['flip_prob'] = 0.5

    def __call__(self, meta_data, img, mask_miss):

        # ランダムにオフセットを用意 -40から40

        dice = random.random()  # (0,1)
        doflip = dice <= self.params_transform['flip_prob']

        if doflip:
            img = img.copy()
            cv2.flip(src=img, flipCode=1, dst=img)
            w = img.shape[1]  # img.shape=（幅、高さ）

            mask_miss = mask_miss.copy()
            cv2.flip(src=mask_miss, flipCode=1, dst=mask_miss)

            '''
            The order in this work:
                (0-'nose'   1-'neck' 2-'right_shoulder' 3-'right_elbow' 4-'right_wrist'
                5-'left_shoulder' 6-'left_elbow'        7-'left_wrist'  8-'right_hip'  
                9-'right_knee'   10-'right_ankle'   11-'left_hip'   12-'left_knee' 
                13-'left_ankle'  14-'right_eye'     15-'left_eye'   16-'right_ear' 
                17-'left_ear' )
            '''
            meta_data['objpos'][0] = w - 1 - meta_data['objpos'][0]
            meta_data['joint_self'][:, 0] = w - \
                1 - meta_data['joint_self'][:, 0]
            # print meta['joint_self']
            meta_data['joint_self'] = meta_data['joint_self'][[0, 1, 5, 6,
                                                               7, 2, 3, 4, 11, 12, 13, 8, 9, 10, 15, 14, 17, 16]]

            num_other_people = meta_data['numOtherPeople']
            if (num_other_people != 0):
                meta_data['objpos_other'][:, 0] = w - \
                    1 - meta_data['objpos_other'][:, 0]
                meta_data['joint_others'][:, :, 0] = w - \
                    1 - meta_data['joint_others'][:, :, 0]
                for i in range(num_other_people):
                    meta_data['joint_others'][i] = meta_data['joint_others'][i][[
                        0, 1, 5, 6, 7, 2, 3, 4, 11, 12, 13, 8, 9, 10, 15, 14, 17, 16]]

        return meta_data, img, mask_miss


class remove_illegal_joint(object):
    """データオーギュメンテーションの結果、画像内から外れたパーツの位置情報を変更する"""

    def __init__(self):
        self.params_transform = dict()
        self.params_transform['crop_size_x'] = 368
        self.params_transform['crop_size_y'] = 368

    def __call__(self, meta_data, img, mask_miss):
        crop_x = int(self.params_transform['crop_size_x'])
        crop_y = int(self.params_transform['crop_size_y'])

        # 条件式4つのORを計算する
        mask = np.logical_or.reduce((meta_data['joint_self'][:, 0] >= crop_x,
                                     meta_data['joint_self'][:, 0] < 0,
                                     meta_data['joint_self'][:, 1] >= crop_y,
                                     meta_data['joint_self'][:, 1] < 0))

        # 画像内の枠からはみ出たパーツの位置情報は(1,1,2)にする
        meta_data['joint_self'][mask == True, :] = (1, 1, 2)

        if (meta_data['numOtherPeople'] != 0):
            mask = np.logical_or.reduce((meta_data['joint_others'][:, :, 0] >= crop_x,
                                         meta_data['joint_others'][:,
                                                                   :, 0] < 0,
                                         meta_data['joint_others'][:,
                                                                   :, 1] >= crop_y,
                                         meta_data['joint_others'][:, :, 1] < 0))
            meta_data['joint_others'][mask == True, :] = (1, 1, 2)

        return meta_data, img, mask_miss


class Normalize_Tensor(object):
    def __init__(self):
        self.color_mean = [0.485, 0.456, 0.406]
        self.color_std = [0.229, 0.224, 0.225]

    def __call__(self, meta_data, img, mask_miss):

        # 画像の大きさは最大1に規格化される
        img = img.astype(np.float32) / 255.

        # 色情報の標準化
        preprocessed_img = img.copy()[:, :, ::-1]  # BGR→RGB

        for i in range(3):
            preprocessed_img[:, :, i] = preprocessed_img[:,
                                                         :, i] - self.color_mean[i]
            preprocessed_img[:, :, i] = preprocessed_img[:,
                                                         :, i] / self.color_std[i]

        # （幅、高さ、色）→（色、幅、高さ）
        img = preprocessed_img.transpose((2, 0, 1)).astype(np.float32)
        mask_miss = mask_miss.transpose((2, 0, 1)).astype(np.float32)

        # 画像をTensorに
        img = torch.from_numpy(img)
        mask_miss = torch.from_numpy(mask_miss)

        return meta_data, img, mask_miss


class no_Normalize_Tensor(object):
    def __init__(self):
        self.color_mean = [0, 0, 0]
        self.color_std = [1, 1, 1]

    def __call__(self, meta_data, img, mask_miss):

        # 画像の大きさは最大1に規格化される
        img = img.astype(np.float32) / 255.

        # 色情報の標準化
        preprocessed_img = img.copy()[:, :, ::-1]  # BGR→RGB

        for i in range(3):
            preprocessed_img[:, :, i] = preprocessed_img[:,
                                                         :, i] - self.color_mean[i]
            preprocessed_img[:, :, i] = preprocessed_img[:,
                                                         :, i] / self.color_std[i]

        # （幅、高さ、色）→（色、幅、高さ）
        img = preprocessed_img.transpose((2, 0, 1)).astype(np.float32)
        mask_miss = mask_miss.transpose((2, 0, 1)).astype(np.float32)

        # 画像をTensorに
        img = torch.from_numpy(img)
        mask_miss = torch.from_numpy(mask_miss)

        return meta_data, img, mask_miss

In [ ]:
class DataTransform():
    def __init__(self):

        self.data_transform = {
            'train': Compose([
                get_anno(),  # JSONからアノテーションを辞書に格納
                add_neck(),  # アノテーションデータの順番を変更し、さらに首のアノテーションデータを追加
                aug_scale(),  # 拡大縮小
                aug_rotate(),  # 回転
                aug_croppad(),  # 切り出し
                aug_flip(),  # 左右反転
                remove_illegal_joint(),  # 画像からはみ出たアノテーションを除去
                # Normalize_Tensor()  # 色情報の標準化とテンソル化
                no_Normalize_Tensor()  # 本節のみ、色情報の標準化をなくす
            ]),
            'val': Compose([
                # 本書では検証は省略
            ])
        }

    def __call__(self, phase, meta_data, img, mask_miss):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        meta_data, img, mask_miss = self.data_transform[phase](
            meta_data, img, mask_miss)

        return meta_data, img, mask_miss

In [ ]:
# 動作確認
# 画像読み込み
index = 24
img = cv2.imread(val_img_list[index])
mask_miss = cv2.imread(val_mask_list[index])
meat_data = val_meta_list[index]

# 画像前処理
transform = DataTransform()
meta_data, img, mask_miss = transform("train", meat_data, img, mask_miss)

# 画像表示
img = img.numpy().transpose((1, 2, 0))
plt.imshow(img)
plt.show()

# マスク表示
mask_miss = mask_miss.numpy().transpose((1, 2, 0))
plt.imshow(mask_miss)
plt.show()

# 合成 RGBにそろえてから
img = Image.fromarray(np.uint8(img*255))
img = np.asarray(img.convert('RGB'))
mask_miss = Image.fromarray(np.uint8((mask_miss)))
mask_miss = np.asarray(mask_miss.convert('RGB'))
blend_img = cv2.addWeighted(img, 0.4, mask_miss, 0.6, 0)
plt.imshow(blend_img)
plt.show()


訓練データの正解情報として使うアノテーションデータの作成

In [ ]:
from PIL import Image
import cv2
import numpy as np
from scipy import misc, ndimage
import torch
import torch.utils.data as data

def putGaussianMaps(center, accumulate_confid_map, params_transform):
    '''ガウスマップに変換する'''
    crop_size_y = params_transform['crop_size_y']
    crop_size_x = params_transform['crop_size_x']
    stride = params_transform['stride']
    sigma = params_transform['sigma']

    grid_y = crop_size_y / stride
    grid_x = crop_size_x / stride
    start = stride / 2.0 - 0.5
    y_range = [i for i in range(int(grid_y))]
    x_range = [i for i in range(int(grid_x))]
    xx, yy = np.meshgrid(x_range, y_range)
    xx = xx * stride + start
    yy = yy * stride + start
    d2 = (xx - center[0]) ** 2 + (yy - center[1]) ** 2
    exponent = d2 / 2.0 / sigma / sigma
    mask = exponent <= 4.6052
    cofid_map = np.exp(-exponent)
    cofid_map = np.multiply(mask, cofid_map)
    accumulate_confid_map += cofid_map
    accumulate_confid_map[accumulate_confid_map > 1.0] = 1.0

    return accumulate_confid_map


def putVecMaps(centerA, centerB, accumulate_vec_map, count, params_transform):
    '''Parts A Fieldのベクトルを求める'''

    centerA = centerA.astype(float)
    centerB = centerB.astype(float)

    stride = params_transform['stride']
    crop_size_y = params_transform['crop_size_y']
    crop_size_x = params_transform['crop_size_x']
    grid_y = crop_size_y / stride
    grid_x = crop_size_x / stride
    thre = params_transform['limb_width']   # limb width
    centerB = centerB / stride
    centerA = centerA / stride

    limb_vec = centerB - centerA
    norm = np.linalg.norm(limb_vec)
    if (norm == 0.0):
        # print 'limb is too short, ignore it...'
        return accumulate_vec_map, count
    limb_vec_unit = limb_vec / norm
    # print 'limb unit vector: {}'.format(limb_vec_unit)

    # To make sure not beyond the border of this two points
    min_x = max(int(round(min(centerA[0], centerB[0]) - thre)), 0)
    max_x = min(int(round(max(centerA[0], centerB[0]) + thre)), grid_x)
    min_y = max(int(round(min(centerA[1], centerB[1]) - thre)), 0)
    max_y = min(int(round(max(centerA[1], centerB[1]) + thre)), grid_y)

    range_x = list(range(int(min_x), int(max_x), 1))
    range_y = list(range(int(min_y), int(max_y), 1))
    xx, yy = np.meshgrid(range_x, range_y)
    ba_x = xx - centerA[0]  # the vector from (x,y) to centerA
    ba_y = yy - centerA[1]
    limb_width = np.abs(ba_x * limb_vec_unit[1] - ba_y * limb_vec_unit[0])
    mask = limb_width < thre  # mask is 2D

    vec_map = np.copy(accumulate_vec_map) * 0.0
    vec_map[yy, xx] = np.repeat(mask[:, :, np.newaxis], 2, axis=2)
    vec_map[yy, xx] *= limb_vec_unit[np.newaxis, np.newaxis, :]

    mask = np.logical_or.reduce(
        (np.abs(vec_map[:, :, 0]) > 0, np.abs(vec_map[:, :, 1]) > 0))

    accumulate_vec_map = np.multiply(
        accumulate_vec_map, count[:, :, np.newaxis])
    accumulate_vec_map += vec_map
    count[mask == True] += 1

    mask = count == 0

    count[mask == True] = 1

    accumulate_vec_map = np.divide(accumulate_vec_map, count[:, :, np.newaxis])
    count[mask == True] = 0

    return accumulate_vec_map, count


def get_ground_truth(meta, mask_miss):
    """アノテーションとマスクデータから正しい答えを求める"""

    # 初期設定
    params_transform = dict()
    params_transform['stride'] = 8  # 画像サイズを変更したくない場合は1にする
    params_transform['mode'] = 5
    params_transform['crop_size_x'] = 368
    params_transform['crop_size_y'] = 368
    params_transform['np'] = 56
    params_transform['sigma'] = 7.0
    params_transform['limb_width'] = 1.0

    stride = params_transform['stride']
    mode = params_transform['mode']
    crop_size_y = params_transform['crop_size_y']
    crop_size_x = params_transform['crop_size_x']
    num_parts = params_transform['np']
    nop = meta['numOtherPeople']

    # 画像サイズ
    grid_y = crop_size_y / stride
    grid_x = crop_size_x / stride
    channels = (num_parts + 1) * 2

    # 格納する変数
    heatmaps = np.zeros((int(grid_y), int(grid_x), 19))
    pafs = np.zeros((int(grid_y), int(grid_x), 38))

    mask_miss = cv2.resize(mask_miss, (0, 0), fx=1.0 / stride, fy=1.0 /
                           stride, interpolation=cv2.INTER_CUBIC).astype(
        np.float32)
    mask_miss = mask_miss / 255.
    mask_miss = np.expand_dims(mask_miss, axis=2)

    # マスク変数
    heat_mask = np.repeat(mask_miss, 19, axis=2)
    paf_mask = np.repeat(mask_miss, 38, axis=2)

    # ピンポイントの座標情報をガウス分布にぼやっとさせる
    for i in range(18):
        if (meta['joint_self'][i, 2] <= 1):
            center = meta['joint_self'][i, :2]
            gaussian_map = heatmaps[:, :, i]
            heatmaps[:, :, i] = putGaussianMaps(
                center, gaussian_map, params_transform)
        for j in range(nop):
            if (meta['joint_others'][j, i, 2] <= 1):
                center = meta['joint_others'][j, i, :2]
                gaussian_map = heatmaps[:, :, i]
                heatmaps[:, :, i] = putGaussianMaps(
                    center, gaussian_map, params_transform)
    # pafs
    mid_1 = [2, 9, 10, 2, 12, 13, 2, 3, 4,
             3, 2, 6, 7, 6, 2, 1, 1, 15, 16]

    mid_2 = [9, 10, 11, 12, 13, 14, 3, 4, 5,
             17, 6, 7, 8, 18, 1, 15, 16, 17, 18]

    thre = 1
    for i in range(19):
        # limb

        count = np.zeros((int(grid_y), int(grid_x)), dtype=np.uint32)
        if (meta['joint_self'][mid_1[i] - 1, 2] <= 1 and meta['joint_self'][mid_2[i] - 1, 2] <= 1):
            centerA = meta['joint_self'][mid_1[i] - 1, :2]
            centerB = meta['joint_self'][mid_2[i] - 1, :2]
            vec_map = pafs[:, :, 2 * i:2 * i + 2]
            #                    print vec_map.shape

            pafs[:, :, 2 * i:2 * i + 2], count = putVecMaps(centerA=centerA,
                                                            centerB=centerB,
                                                            accumulate_vec_map=vec_map,
                                                            count=count, params_transform=params_transform)
        for j in range(nop):
            if (meta['joint_others'][j, mid_1[i] - 1, 2] <= 1 and meta['joint_others'][j, mid_2[i] - 1, 2] <= 1):
                centerA = meta['joint_others'][j, mid_1[i] - 1, :2]
                centerB = meta['joint_others'][j, mid_2[i] - 1, :2]
                vec_map = pafs[:, :, 2 * i:2 * i + 2]
                pafs[:, :, 2 * i:2 * i + 2], count = putVecMaps(centerA=centerA,
                                                                centerB=centerB,
                                                                accumulate_vec_map=vec_map,
                                                                count=count, params_transform=params_transform)
    # background
    heatmaps[:, :, -
             1] = np.maximum(1 - np.max(heatmaps[:, :, :18], axis=2), 0.)

    # Tensorに
    heat_mask = torch.from_numpy(heat_mask)
    heatmaps = torch.from_numpy(heatmaps)
    paf_mask = torch.from_numpy(paf_mask)
    pafs = torch.from_numpy(pafs)

    return heat_mask, heatmaps, paf_mask, pafs


def make_datapath_list(rootpath):
    """
    学習、検証の画像データとアノテーションデータ、マスクデータへのファイルパスリストを作成する。
    """

    # アノテーションのJSONファイルを読み込む
    json_path = os.path.join(rootpath, 'COCO.json')
    with open(json_path) as data_file:
        data_this = json.load(data_file)
        data_json = data_this['root']

    # indexを格納
    num_samples = len(data_json)
    train_indexes = []
    val_indexes = []
    for count in range(num_samples):
        if data_json[count]['isValidation'] != 0.:
            val_indexes.append(count)
        else:
            train_indexes.append(count)

    # 画像ファイルパスを格納
    train_img_list = list()
    val_img_list = list()

    for idx in train_indexes:
        img_path = os.path.join(rootpath, data_json[idx]['img_paths'])
        train_img_list.append(img_path)

    for idx in val_indexes:
        img_path = os.path.join(rootpath, data_json[idx]['img_paths'])
        val_img_list.append(img_path)

    # マスクデータのパスを格納
    train_mask_list = []
    val_mask_list = []

    for idx in train_indexes:
        img_idx = data_json[idx]['img_paths'][-16:-4]
        anno_path = "./data/mask/train2014/mask_COCO_train2014_" + img_idx+'.jpg'
        train_mask_list.append(anno_path)

    for idx in val_indexes:
        img_idx = data_json[idx]['img_paths'][-16:-4]
        anno_path = "./data/mask/val2014/mask_COCO_val2014_" + img_idx+'.jpg'
        val_mask_list.append(anno_path)

    # アノテーションデータを格納
    train_meta_list = list()
    val_meta_list = list()

    for idx in train_indexes:
        train_meta_list.append(data_json[idx])

    for idx in val_indexes:
        val_meta_list.append(data_json[idx])

    return train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list


class DataTransform():
    """
    画像とマスク、アノテーションの前処理クラス。
    学習時と推論時で異なる動作をする。
    学習時はデータオーギュメンテーションする。
    """

    def __init__(self):

        self.data_transform = {
            'train': Compose([
                get_anno(),  # JSONからアノテーションを辞書に格納
                add_neck(),  # アノテーションデータの順番を変更し、さらに首のアノテーションデータを追加
                aug_scale(),  # 拡大縮小
                aug_rotate(),  # 回転
                aug_croppad(),  # 切り出し
                aug_flip(),  # 左右反転
                remove_illegal_joint(),  # 画像からはみ出たアノテーションを除去
                Normalize_Tensor()  # 色情報の標準化とテンソル化
                # no_Normalize_Tensor()  # 本節のみ、色情報の標準化をなくす
            ]),
            'val': Compose([
                # 本書では検証は省略
            ])
        }

    def __call__(self, phase, meta_data, img, mask_miss):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        meta_data, img, mask_miss = self.data_transform[phase](
            meta_data, img, mask_miss)

        return meta_data, img, mask_miss


class COCOkeypointsDataset(data.Dataset):
    """
    MSCOCOのCocokeypointsのDatasetを作成するクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    img_list : リスト
        画像のパスを格納したリスト
    anno_list : リスト
        アノテーションへのパスを格納したリスト
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    transform : object
        前処理クラスのインスタンス

    """

    def __init__(self, img_list, mask_list, meta_list, phase, transform):
        self.img_list = img_list
        self.mask_list = mask_list
        self.meta_list = meta_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.img_list)

    def __getitem__(self, index):
        img, heatmaps, heat_mask, pafs, paf_mask = self.pull_item(index)
        return img, heatmaps, heat_mask, pafs, paf_mask

    def pull_item(self, index):
        '''画像のTensor形式のデータ、アノテーション、マスクを取得する'''

        # 1. 画像読み込み
        image_file_path = self.img_list[index]
        img = cv2.imread(image_file_path)  # [高さ][幅][色BGR]

        # 2. マスクとアノテーション読み込み
        mask_miss = cv2.imread(self.mask_list[index])
        meat_data = self.meta_list[index]

        # 3. 画像前処理
        meta_data, img, mask_miss = self.transform(
            self.phase, meat_data, img, mask_miss)

        # 4. 正解アノテーションデータの取得
        mask_miss_numpy = mask_miss.numpy().transpose((1, 2, 0))
        heat_mask, heatmaps, paf_mask, pafs = get_ground_truth(
            meta_data, mask_miss_numpy)

        # 5. マスクデータはRGBが(1,1,1)か(0,0,0)なので、次元を落とす
        heat_mask = heat_mask[:, :, :, 0]
        paf_mask = paf_mask[:, :, :, 0]

        # 6. チャネルが最後尾にあるので順番を変える
        # 例：paf_mask：torch.Size([46, 46, 38])
        # → torch.Size([38, 46, 46])
        paf_mask = paf_mask.permute(2, 0, 1)
        heat_mask = heat_mask.permute(2, 0, 1)
        pafs = pafs.permute(2, 0, 1)
        heatmaps = heatmaps.permute(2, 0, 1)

        return img, heatmaps, heat_mask, pafs, paf_mask

In [ ]:
# 画像読み込み
index = 24
img = cv2.imread(val_img_list[index])
mask_miss = cv2.imread(val_mask_list[index])
meat_data = val_meta_list[index]

# 画像前処理
meta_data, img, mask_miss = transform("train", meat_data, img, mask_miss)

img = img.numpy().transpose((1, 2, 0))
mask_miss = mask_miss.numpy().transpose((1, 2, 0))

# OpenPoseのアノテーションデータ生成
heat_mask, heatmaps, paf_mask, pafs = get_ground_truth(meta_data, mask_miss)

# 画像表示
plt.imshow(img)
plt.show()

In [ ]:
class COCOkeypointsDataset(data.Dataset):
    def __init__(self, img_list, mask_list, meta_list, phase, transform):
        self.img_list = img_list
        self.mask_list = mask_list
        self.meta_list = meta_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.img_list)

    def __getitem__(self, index):
        img, heatmaps, heat_mask, pafs, paf_mask = self.pull_item(index)
        return img, heatmaps, heat_mask, pafs, paf_mask

    def pull_item(self, index):
        '''画像のTensor形式のデータ、アノテーション、マスクを取得する'''

        # 1. 画像読み込み
        image_file_path = self.img_list[index]
        img = cv2.imread(image_file_path)  # [高さ][幅][色BGR]

        # 2. マスクとアノテーション読み込み
        mask_miss = cv2.imread(self.mask_list[index])
        meat_data = self.meta_list[index]

        # 3. 画像前処理
        meta_data, img, mask_miss = self.transform(
            self.phase, meat_data, img, mask_miss)

        # 4. 正解アノテーションデータの取得
        mask_miss_numpy = mask_miss.numpy().transpose((1, 2, 0))
        heat_mask, heatmaps, paf_mask, pafs = get_ground_truth(
            meta_data, mask_miss_numpy)

        # 5. マスクデータはRGBが(1,1,1)か(0,0,0)なので、次元を落とす
        # マスクデータはマスクされている場所は値が0、それ以外は値が1です
        heat_mask = heat_mask[:, :, :, 0]
        paf_mask = paf_mask[:, :, :, 0]

        # 6. チャネルが最後尾にあるので順番を変える
        # 例：paf_mask：torch.Size([46, 46, 38])
        # → torch.Size([38, 46, 46])
        paf_mask = paf_mask.permute(2, 0, 1)
        heat_mask = heat_mask.permute(2, 0, 1)
        pafs = pafs.permute(2, 0, 1)
        heatmaps = heatmaps.permute(2, 0, 1)

        return img, heatmaps, heat_mask, pafs, paf_mask

In [ ]:
# 動作確認
train_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())
val_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())


# データの取り出し例
item = train_dataset.__getitem__(0)
print(item[0].shape)  # img
print(item[1].shape)  # heatmaps,
print(item[2].shape)  # heat_mask
print(item[3].shape)  # pafs 
print(item[4].shape)  # paf_mask

DataLoaderの作成

In [ ]:
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 動作の確認
batch_iterator = iter(dataloaders_dict["train"])  # イタレータに変換
item = next(batch_iterator)  # 1番目の要素を取り出す
print(item[0].shape)  # img
print(item[1].shape)  # heatmaps,
print(item[2].shape)  # heat_mask
print(item[3].shape)  # pafs 
print(item[4].shape)  # paf_mask